In [1]:
# load txt file
with open('/home/ctang/ttmp/MMCSR/dataset_metadata/100_way_metadata.txt', 'r') as f:
    metadata = f.readlines()
metadata = [x.strip() for x in metadata]

In [13]:
embeddings = np.array([])

In [14]:
from typing import Dict

import torch
import pickle
import numpy as np
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm
# from sentence_transformers.util import cos_sim

# For retrieval you need to pass this prompt. Please find our more in our blog post.
def transform_query(query: str) -> str:
    """ For retrieval, add the prompt for query (not for documents).
    """
    return f'Represent this sentence for searching relevant passages: {query}'

# The model works really well with cls pooling (default) but also with mean pooling.
def pooling(outputs: torch.Tensor, inputs: Dict,  strategy: str = 'cls') -> np.ndarray:
    if strategy == 'cls':
        outputs = outputs[:, 0]
    elif strategy == 'mean':
        outputs = torch.sum(
            outputs * inputs["attention_mask"][:, :, None], dim=1) / torch.sum(inputs["attention_mask"])
    else:
        raise NotImplementedError
    return outputs.detach().cpu().numpy()

# 1. load model
model_id = 'mixedbread-ai/mxbai-embed-large-v1'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)


# encode in batches with tqdm

batch_size = 2
for i in tqdm(range(0, len(metadata), batch_size)):
    docs = metadata[i:i+batch_size]
    inputs = tokenizer(docs, padding=True, return_tensors='pt')
    for k, v in inputs.items():
        inputs[k] = v.to(model.device)
    outputs = model(**inputs).last_hidden_state
    new_embeddings = pooling(outputs, inputs, 'cls')
    embeddings = np.vstack([embeddings, new_embeddings]) if embeddings.size else new_embeddings

100%|██████████| 2783/2783 [09:41<00:00,  4.79it/s]


In [15]:
# save embeddings
with open('/home/ctang/ttmp/MMCSR/dataset_metadata/100_way_metadata_embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)